In [1]:
import matplotlib
import pathlib
import mne
import os
import numpy as np

In [2]:
# LOAD RAW:
matplotlib.use('Qt5Agg')
data_dir = 'D:/raw_/original data/Control'
set_files = [f for f in os.listdir(data_dir) if f.endswith('.set')]

for set_file in set_files:
    set_path = os.path.join(data_dir,set_file)
    raw = mne.io.read_raw_eeglab(set_path)
    print(f"Loaded data from {set_path}")

Reading D:\raw_\original data\Control\1_sub001.fdt
Loaded data from D:/raw_/original data/Control\1_sub001.set
Reading D:\raw_\original data\Control\1_sub004.fdt
Loaded data from D:/raw_/original data/Control\1_sub004.set
Reading D:\raw_\original data\Control\1_sub006.fdt
Loaded data from D:/raw_/original data/Control\1_sub006.set
Reading D:\raw_\original data\Control\1_sub007.fdt
Loaded data from D:/raw_/original data/Control\1_sub007.set
Reading D:\raw_\original data\Control\1_sub009.fdt
Loaded data from D:/raw_/original data/Control\1_sub009.set
Reading D:\raw_\original data\Control\1_sub012.fdt
Loaded data from D:/raw_/original data/Control\1_sub012.set
Reading D:\raw_\original data\Control\1_sub014.fdt
Loaded data from D:/raw_/original data/Control\1_sub014.set
Reading D:\raw_\original data\Control\1_sub015.fdt
Loaded data from D:/raw_/original data/Control\1_sub015.set
Reading D:\raw_\original data\Control\1_sub016.fdt
Loaded data from D:/raw_/original data/Control\1_sub016.set
R

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3512\390603970.py:8: RuntimeWarning: Data file name in EEG.data (1_sub0012.fdt) is incorrect, the file name must have changed on disk, using the correct file name (1_sub012.fdt).
  raw = mne.io.read_raw_eeglab(set_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3512\390603970.py:8: RuntimeWarning: Data file name in EEG.data (1_sub0014.fdt) is incorrect, the file name must have changed on disk, using the correct file name (1_sub014.fdt).
  raw = mne.io.read_raw_eeglab(set_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3512\390603970.py:8: RuntimeWarning: Data file name in EEG.data (1_sub0015.fdt) is incorrect, the file name must have changed on disk, using the correct file name (1_sub015.fdt).
  raw = mne.io.read_raw_eeglab(set_path)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3512\390603970.py:8: RuntimeWarning: Data file name in EEG.data (1_sub0016.fdt) is incorrect, the file name must have changed on dis

In [3]:
# 为了计算后续crop长度&resampling目标频率，读取
freq = raw.info['sfreq']
n_samples = raw.n_times/freq
n_samples # 采样总数

1321.0

In [4]:
# Delete non-eeg channels

non_eeg_chan = ['ECG',
 'HEOR',
 'HEOL',
 'VEOU',
 'VEOL']

raw.drop_channels(non_eeg_chan)

raw.info['ch_names']
raw.plot()

Using qt as 2D backend.


In [5]:
# Read events & crop
events, event_id = mne.events_from_annotations(raw)
# 源数据里没有语义化events，自己用词典
# event_id = {'Openeyes':events[1]['12'],'Closeeyes':events[1]['14']}

# 从后面看，闭眼后应当留出五分钟？?
padding_duration = 300000
t_start = 40550 -5000
t_end = 400626 +padding_duration


# 直接把tuple的描述字典赋值给event_id
# 每隔一秒建立一个event
event_id = 1  # 事件ID
duration = 1  # 事件持续时间/秒
events = mne.make_fixed_length_events(raw, event_id, start=0, duration=duration, overlap=0)
events

Used Annotations descriptions: ['12', '13', '14', '15']


array([[      0,       0,       1],
       [   1000,       0,       1],
       [   2000,       0,       1],
       ...,
       [1318000,       0,       1],
       [1319000,       0,       1],
       [1320000,       0,       1]])

In [6]:
# 创建Epochs对象
reject_criteria = dict(eeg = 100e-6)
# 去极值后，13，14，15都被drop了，而且12里只有一个epoch（总共一秒。。。）
# only one event was found within that time window for ID '12'

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=-1,
                    tmax=1,
                    reject=reject_criteria,
                    preload=True)

raw_cropped = raw.crop(tmin=t_start/1000,tmax=t_end/1000)
raw_cropped.plot()

Not setting metadata
1321 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 1321 events and 2001 original time points ...
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FT8', 'C5', 'T7', 'T8', 'CP4', 'CP5', 'CP6', 'TP7', 'TP8', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'POz', 'PO3', 'PO4', 'PO5', 'PO6', 'PO7', 'PO8', 'Oz', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F3', 'F5', 'F7', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'CP5', 'CP6', 'TP7', 'TP8', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'POz', 'PO4', 'PO5', 'PO6', 'PO7', 'PO8', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF7', 'AF8', 'FT7', 'FT8', 'T7', 'T8', 'CP5', 'TP7', 'TP8', 'P3', 'P5', 'P6', 'P7', 'P8', 'POz', 'PO4', 'PO5', 'PO6', 'PO7', 'PO8', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', '

MemoryError: Unable to allocate 1.16 GiB for an array with shape (1321, 59, 2001) and data type float64

In [7]:
# NO CHANNEL LOCATIONS IN EEG FILE

montage = mne.channels.make_standard_montage("standard_1005")
print(montage)

raw_cropped.set_montage(montage)
fig = raw_cropped.plot_sensors(show_names=True)

#定位完成

<DigMontage | 0 extras (headshape), 0 HPIs, 3 fiducials, 343 channels>


NameError: name 'raw_cropped' is not defined

In [ ]:
# resampling
target_sfreq = 500

raw_resampled = raw_cropped.copy().resample(sfreq=target_sfreq)
raw_resampled.plot_psd(fmax=250)

In [ ]:
raw_resampled.plot()

In [ ]:
# filtering
raw_filtered=raw_resampled.copy().filter(l_freq=1,h_freq=100)
raw_filtered
raw_filtered.plot()
raw_filtered.plot_psd(fmax=250)

In [ ]:
# ICA
raw_ica = raw_filtered.copy()
ica = mne.preprocessing.ICA(n_components= 20)
ica.fit(raw_ica)
raw_ica.load_data()
ica.plot_sources(raw_ica)
ica.plot_components()

In [ ]:
# re-ref based on average channel(?
raw_ref = raw_ica.copy()
raw_ref.load_data()
raw_ref.set_eeg_reference(ref_channels="average", projection=False)

raw_ref.plot()

In [ ]:
epochs.plot()

In [ ]:
raw_ref.plot_psd()

In [ ]:
save_dir='D:/raw_/results/control'
epochs.save(save_dir)